## Based on 15 years (2002-2017) of power outage data, predict highest outage event type in 2018/2017, before an electric disturbance is reported so as to prevent aftermath effects and prepared and have minimal effects. 

Develop a prediction model in Python for power outage detection using supervised learning. 


### 1. Create empty dataframe object



In [164]:
import pandas as pd
import calendar

combined = pd.DataFrame()

## 2. Download Electric disturbances annual summaries from [here](http://www.oe.netl.doe.gov/OE417_annual_summary.aspx)

Note: You can download clean data from [here (Google Spreadsheet)](http://insideenergy.org/2014/08/18/data-explore-15-years-of-power-outages/) for 2002-2014 or [CSV](https://docs.google.com/spreadsheets/d/1AdxhulfM9jeqviIZihuODqk7HoS1kRUlM_afIKXAjXQ/edit#gid=595041757). 

## 3. Check for data inconsistencies



#### Note: When reading file with read_excel this message is displayed:  WARNING:  *** file size (48323) not 512 + multiple of sector size (512).
To avoid this warning, open downloaded files in MS Excel, enable editing and save.

In [133]:
for year in range(2002, 2018):
        df = ""        
        file_path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)
        try:
            df = pd.read_excel(file_path)
            
            print ("{0} : {1}, isnull: {2}".format(year, len(df.columns), 
                                                   df.isnull().values.any()))
        except Exception as e:
            print(e)        

2002 : 8, isnull: True
2003 : 8, isnull: True
2004 : 8, isnull: True
2005 : 8, isnull: True
2006 : 8, isnull: True
2007 : 8, isnull: True
2008 : 8, isnull: True
2009 : 8, isnull: True
2010 : 8, isnull: True
2011 : 9, isnull: True
2012 : 9, isnull: True
2013 : 9, isnull: True
2014 : 9, isnull: True
2015 : 11, isnull: True
2016 : 11, isnull: True
2017 : 11, isnull: False


Note: The column number and null data varies by year. So group files by same number of columns and prepare excel file accordingly.

# 3. Clean Data to make it conistent

Based on the number of columns from above files will be grouped into :
* 2002 to 2010
* 2011 to 2014
* 2015 to 2017





### 3.1 Clean data from 2002 - 2010

### Observe data

In [359]:
for year in range(2002, 2011):
        df = ""        
        file_path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)
        try:
            df = pd.read_excel(file_path, header=1)
            
            print ("{0} : {1}, isnull: {2}".format(year, len(df.columns), 
                                                   df.isnull().values.any()))
            print (df.columns)
        except Exception as e:
            print(e)        

2002 : 8, isnull: True
Index(['Date', 'NERC Region', 'Time', 'Area', 'Type of Disturbance',
       'Loss (megawatts)', 'Number of Customers Affected', 'Restoration Time'],
      dtype='object')
2003 : 8, isnull: True
Index(['Date', 'NERC Region', 'Time', 'Area Affected', 'Type of Disturbance',
       'Loss (megawatts)', 'Number of Customers Affected 1', 'Restoration'],
      dtype='object')
2004 : 8, isnull: True
Index(['Date', 'NERC Region', 'Time', 'Area Affected', 'Type of Disturbance',
       'Loss (megawatts)', 'Number of Customers Affected 1', 'Restoration'],
      dtype='object')
2005 : 8, isnull: True
Index(['Date', 'NERC Region', 'Time', 'Area Affected', 'Type of Disturbance',
       'Loss (megawatts)', 'Number of Customers Affected 1', 'Restoration'],
      dtype='object')
2006 : 8, isnull: True
Index(['Date', 'NERC Region', 'Time', 'Area Affected', 'Type of Disturbance',
       'Loss (megawatts)', 'Number of Customers Affected 1', 'Restoration'],
      dtype='object')
2007 :

### Observations
1. Column 'NERC Region' is consistent among all years. However, in 2007 the column name is  ' NERC Region', notice the trailing space at the beginning. This will be avoided by setting 'names' property to a standard list of column names. 
2. In 2008 the first two rows must be skipped because of unnecessary text. 
3. Rest of the years only first row is skipped hence header index - 1

### Steps to clean data:

1. Read excel file with the following attributes 
    1. Set header to 2 if year = 2008 otherwise 1.  
    2. Set names  to a list of 8 column names  
2. Drop rows that don't have 'Type of Disturbance'
3. Drop columns that have all NaN  
4. Optional: add new column 'Restoration Date' to split 'Restoration Time' into date and time


In [335]:
combined_1 = pd.DataFrame()
for year in range(2002, 2011):
        df1 = ""        
        file_path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)
        try:
            h = 1
            if year == 2008:
                h = 2
            df1 = pd.read_excel(file_path,header=h, 
                               names=['Date Event Began', 'NERC Region',
                                     'Time Event Began', 'Area affected', 
                                     'Disturbance Type', 'Demand Loss(MW)',
                                     'Customers affected', 'Restoration Time'])
            # Drop rows where column 'Type of Disturbance' is NaN
            df1 = df1.dropna(axis=0,how='all',subset=['Type of Disturbance'])
            print (df1.shape)
            df1 = df1.dropna(axis=1,how='all')
            print (df1.shape)
            print ("{0} : {1}, isnull: {2}".format(year, len(df1.columns), 
                                                   df1.isnull().values.any()))
            # Add column Restoration Date to the dataframe
            df1['Restoration Date'] = ''
            print(df1.columns)
        except Exception as e:
            print(e)        
        combined_1 = combined_1.append(df1, ignore_index=True)        

(24, 8)
(24, 8)
2002 : 8, isnull: True
Index(['Date Event Began', 'NERC Region', 'Time Event Began', 'Area affected',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected',
       'Restoration Time'],
      dtype='object')
(64, 8)
(64, 8)
2003 : 8, isnull: True
Index(['Date Event Began', 'NERC Region', 'Time Event Began', 'Area affected',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected',
       'Restoration Time'],
      dtype='object')
(97, 8)
(97, 8)
2004 : 8, isnull: False
Index(['Date Event Began', 'NERC Region', 'Time Event Began', 'Area affected',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected',
       'Restoration Time'],
      dtype='object')
(89, 8)
(89, 8)
2005 : 8, isnull: True
Index(['Date Event Began', 'NERC Region', 'Time Event Began', 'Area affected',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected',
       'Restoration Time'],
      dtype='object')
(94, 8)
(94, 8)
2006 : 8, isnull: True
Inde

In [336]:
combined_1.shape

(827, 8)

In [289]:
combined_1.head()


,Date Event Began,NERC Region,Time Event Began,Area affected,Type of Disturbance,Demand Loss(MW),Customers affected,Restoration Time
0,2002-01-30 00:00:00,SPP,06:00:00,Oklahoma,Ice Storm,500,1881134,2002-02-07 12:00:00
1,2002-01-29 00:00:00,SPP,Evening,Metropolitan Kansas City Area,Ice Storm,500-600,270000,NaN
2,2002-01-30 00:00:00,SPP,16:00:00,Missouri,Ice Storm,210,95000,2002-02-10 21:00:00
3,2002-02-27 00:00:00,WSCC,10:48:00,California,Interruption of Firm Load,300,255000,2002-02-27 11:35:00
4,2002-03-09 00:00:00,ECAR,00:00:00,Lower Peninsula of Michigan,Severe Weather,190,190000,2002-03-11 12:00:00


### 3.2 Clean data from 2011 - 2014

### Observe data

In [360]:
for year in range(2011, 2015):
        df = ""        
        file_path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)
        try:
            df = pd.read_excel(file_path, header=1)
            
            print ("{0} : {1}, isnull: {2}".format(year, len(df.columns), 
                                                   df.isnull().values.any()))
            print (df.columns)
        except Exception as e:
            print(e)        

2011 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Event Type',
       'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')
2012 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Event Type',
       'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')
2013 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Event Type',
       'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')
2014 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Event Type',
       'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='objec

### Observations

1. This group has 9 columns, which is what we're aiming for
2. No missing or unecessary columns
3. Data looks clean

## Steps to clean data:

1. Read excel file with the following attributes 
    1. Set header to 1  
    2. Set names  to a list of 9 column names  
2. Drop rows that don't have 'Type of Disturbance'
3. Drop columns that have all NaN  

In [333]:
combined_2 = pd.DataFrame()
for year in range(2011, 2015):
        df2 = ""        
        path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)        
        try:
            df2 = pd.read_excel(path, header=1,
                               names=['Date Event Began', 
                                     'Time Event Began',
                                      'Restoration Date','Restoration Time',
                                      'Area affected', 'NERC Region',
                                     'Disturbance Type', 'Demand Loss(MW)',
                                     'Customers affected'])
            # Drop rows where column 'Type of Disturbance' is NaN
            df2 = df2.dropna(axis=0,how='all',subset=['Type of Disturbance'])
            print (df2.shape)
            df2 = df2.dropna(axis=1,how='all')
            print (df2.shape)
            print ("{0} : {1}, isnull: {2}".format(year, len(df2.columns), 
                                                   df2.isnull().values.any()))
            print(df2.columns)
        except Exception as e:
            print(e)        
        combined_2 = combined_2.append(df2, ignore_index=True)        

(307, 9)
(307, 9)
2011 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')
(203, 9)
(203, 9)
2012 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')
(174, 9)
(174, 9)
2013 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')
(214, 9)
(214, 9)
2014 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', '

In [334]:
combined_2.shape

(898, 9)

In [308]:
combined_2.tail()

,Date Event Began,Time Event Began,Restoration Date,Restoration Time,Area affected,NERC Region,Type of Disturbance,Demand Loss(MW),Customers affected
893,2014-12-11 00:00:00,16:05:00,2014-12-11 00:00:00,21:00:00,"Portland, Oregon",WECC,Severe Weather- High Winds,250,85470
894,2014-12-11 00:00:00,17:00:00,2014-12-12 00:00:00,10:00:00,"Kitsap, Thurston, Whatcom counties Washington",WECC,Severe Weather- High Winds,116,264000
895,2014-12-17 00:00:00,11:00:00,2014-12-17 00:00:00,12:15:00,Washington,WECC,Suspected Physical Attack,Unknown,Unknown
896,2014-12-30 00:00:00,15:50:00,2014-12-31 00:00:00,11:00:00,"New Hampshire, Massachusetts, Maine, Rhode Isl...",NPCC,Suspected Cyber Attack,Unknown,Unknown
897,2014-12-30 00:00:00,13:08:00,2015-01-01 00:00:00,16:50:00,Northern California,WECC,Severe Weather- High Winds,127,84500


### 3.3 Clean data from 2015 - 2017

#### Observe data

In [355]:
for year in range(2015, 2018):
        df = ""        
        file_path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)
        try:
            df = pd.read_excel(file_path,header=1)
            
            print ("{0} : {1}, isnull: {2}".format(year, len(df.columns), 
                                                   df.isnull().values.any()))
            print(df.columns)
        except Exception as e:
            print(e)        

2015 : 11, isnull: True
Index(['Month', 'Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Alert Criteria',
       'Event Type', 'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')
2016 : 11, isnull: True
Index(['Month', 'Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Alert Criteria',
       'Event Type', 'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')
2017 : 11, isnull: False
Index(['Month', 'Date Event Began', 'Time Event Began', 'Date of Restoration',
       'Time of Restoration', 'Area Affected', 'NERC Region', 'Alert Criteria',
       'Event Type', 'Demand Loss (MW)', 'Number of Customers Affected'],
      dtype='object')


### Observations 

1. This group has 11 columns. 2 additional columns 'Month' and 'Alert Criteria' can be dropped.

### Steps to clean data:

1. Read excel file with the following attributes 
    1. Set header to 1
    2. Parse columns other than 'Month' and 'Alert Criteria' 
    3. Set names  to a list of 9 column names
2. Drop rows that don't have 'Type of Disturbance'
3. Drop columns that have all NaN

In [337]:
combined_3 = pd.DataFrame()
for year in range(2015, 2018):
        df3 = ""        
        path = "C:\downloadedfrompython\{0}_Annual_Summary.xls".format(year)        
        try:
            df3 = pd.read_excel(path, header=1,parse_cols='B:G,I:K',
                               names=['Date Event Began', 
                                     'Time Event Began',
                                      'Restoration Date','Restoration Time',
                                      'Area affected', 'NERC Region',
                                     'Disturbance Type', 'Demand Loss(MW)',
                                     'Customers affected'])
            # Drop rows where column 'Type of Disturbance' is NaN
            df3 = df3.dropna(axis=0,how='all',subset=['Type of Disturbance'])
            print (df3.shape)
            df2 = df3.dropna(axis=1,how='all')
            print (df3.shape)
            print ("{0} : {1}, isnull: {2}".format(year, len(df3.columns), 
                                                   df3.isnull().values.any()))
            print(df3.columns)
        except Exception as e:
            print(e)        
        combined_3 = combined_3.append(df3, ignore_index=True)        

(143, 9)
(143, 9)
2015 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')
(141, 9)
(141, 9)
2016 : 9, isnull: True
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')
(49, 9)
(49, 9)
2017 : 9, isnull: False
Index(['Date Event Began', 'Time Event Began', 'Restoration Date',
       'Restoration Time', 'Area affected', 'NERC Region',
       'Type of Disturbance', 'Demand Loss(MW)', 'Customers affected'],
      dtype='object')


In [340]:
combined_3.shape

(333, 9)

In [341]:
combined_3.head(1)

,Date Event Began,Time Event Began,Restoration Date,Restoration Time,Area affected,NERC Region,Type of Disturbance,Demand Loss(MW),Customers affected
0,2015-01-07 00:00:00,17:00:00,2015-01-08 00:00:00,08:35:00,Tennessee,SERC,Severe Weather - Winter,Unknown,Unknown


## 4. Merge data from 2002-2017

In [347]:
combined = pd.DataFrame()
combined = combined.append(combined_1,ignore_index=True)
combined = combined.append(combined_2,ignore_index=True)
combined = combined.append(combined_3,ignore_index=True)
combined.shape

(2058, 9)

In [350]:
combined.head()

,Area affected,Customers affected,Date Event Began,Demand Loss(MW),NERC Region,Restoration Date,Restoration Time,Time Event Began,Type of Disturbance
0,Oklahoma,1881134,2002-01-30 00:00:00,500,SPP,NaN,2002-02-07 12:00:00,06:00:00,Ice Storm
1,Metropolitan Kansas City Area,270000,2002-01-29 00:00:00,500-600,SPP,NaN,NaN,Evening,Ice Storm
2,Missouri,95000,2002-01-30 00:00:00,210,SPP,NaN,2002-02-10 21:00:00,16:00:00,Ice Storm
3,California,255000,2002-02-27 00:00:00,300,WSCC,NaN,2002-02-27 11:35:00,10:48:00,Interruption of Firm Load
4,Lower Peninsula of Michigan,190000,2002-03-09 00:00:00,190,ECAR,NaN,2002-03-11 12:00:00,00:00:00,Severe Weather


### To download this notebook in Latex or pdfLatex install these two libraries from conda